<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/prepare_data/Load_CovidData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal:

This colab loads data for COVID-19 and merges to population data

# Topic: Investigate Covid-19 NY and CA data

This colab include covid case data from JHU and moving avg trend analysis visualization

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=9a3b0ccf206bc60699e22a502b203420a3e4d31717bfe0b8416acd8cf3b20b4c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


## Import Libraries

In [2]:
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Mount Google drive


In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
location = "drive/Shared drives/the-mean-sqaures/the-mean-squares/Colab_Dataset/Dataset/"

In [ ]:
!ls /content/drive/My\ Drive/MasterProject-Personal/data

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data': No such file or directory


# Data Load

## 1 Load County Population


In [5]:
county_population_US = pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)

(3195, 4)


In [6]:
wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv('covid_county_population_usafacts.csv',low_memory=False)
print(county_population_US.shape)




(3195, 4)


In [7]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


## 2 Load Covid-19 case details (until July 12)

In [8]:
!ls '/content/drive/My Drive/MasterProject-Personal/data/'

ls: cannot access '/content/drive/My Drive/MasterProject-Personal/data/': No such file or directory


### Note about data:
John Hopkins university updates data every day hence we are pulling from repository directly

**US Confirmed url** :https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

**US deaths url**: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv

In [11]:
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

files = [wget.download(url) for url in urls]        

In [12]:
confirmed_US = pd.read_csv(files[0],low_memory=False)
death_US = pd.read_csv(files[1],low_memory=False)
print(confirmed_US.shape)
print(death_US.shape)
print(confirmed_US.head(2))
death_US.head(2)

(3340, 364)
(3340, 365)
        UID iso2 iso3  code3    FIPS  ... 1/4/21 1/5/21 1/6/21  1/7/21  1/8/21
0  84001001   US  USA    840  1001.0  ...   4336   4546   4645    4705    4770
1  84001003   US  USA    840  1003.0  ...  14187  14440  14656   14845   15052

[2 rows x 364 columns]


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,...,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,42,42,42,42,42,42,42,42,42,42,42,42,41,41,41,43,43,43,44,44,44,44,44,46,46,46,46,47,47,47,48,48,50,50,50,50,50,50,50,50
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,98,137,137,137,138,138,138,138,141,141,141,141,141,141,141,141,145,145,146,147,147,147,149,151,151,151,151,152,152,156,160,161,169,169,169,169,169,171,171,171


In [13]:
#print(len(mask_data['state_name'].unique()))
print(len(confirmed_US['Province_State'].unique()))
confirmed_US['Province_State'].unique()

58


array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

## Data cleaning

In [14]:
confirmed_US.columns[:11]

# Note: the first 11 columns contains UID, ios2,ios3, code ,FIPS, 'Admin2', 'Province_State',
      # 'Country_Region', 'Lat', 'Long_',Combined_Key', 'Population',


Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
      dtype='object')

In [15]:
# date begins from 11th column
confirmed_dates = confirmed_US.columns[11:]
confirmed_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '12/30/20', '12/31/20', '1/1/21', '1/2/21', '1/3/21', '1/4/21',
       '1/5/21', '1/6/21', '1/7/21', '1/8/21'],
      dtype='object', length=353)

In [16]:
death_US.columns[:12]

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
      dtype='object')

In [17]:
death_US.columns[10:]

Index(['Combined_Key', 'Population', '1/22/20', '1/23/20', '1/24/20',
       '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       ...
       '12/30/20', '12/31/20', '1/1/21', '1/2/21', '1/3/21', '1/4/21',
       '1/5/21', '1/6/21', '1/7/21', '1/8/21'],
      dtype='object', length=355)

In [18]:
death_dates = death_US.columns[12:]
death_dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '12/30/20', '12/31/20', '1/1/21', '1/2/21', '1/3/21', '1/4/21',
       '1/5/21', '1/6/21', '1/7/21', '1/8/21'],
      dtype='object', length=353)

#### Note: both the date values has the same beginning date 1/22/2020. Hence we can use either of the values

In [19]:
confirmed_df_long = confirmed_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key'],
       value_vars=confirmed_dates,
       var_name = 'Date',
       value_name = 'Confirmed'
)

death_df_long = death_US.melt(
    id_vars=['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'],
       value_vars=death_dates,
       var_name = 'Date',
       value_name = 'Deaths'
)

In [20]:
confirmed_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
1179010,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585282,"Park, Wyoming, US",1/8/21,2211
1179011,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",1/8/21,476
1179012,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",1/8/21,2667
1179013,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",1/8/21,618
1179014,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",1/8/21,3153
1179015,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",1/8/21,2346
1179016,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",1/8/21,1693
1179017,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",1/8/21,0
1179018,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",1/8/21,797
1179019,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",1/8/21,485


In [21]:
confirmed_df_long[confirmed_df_long['FIPS'] == 36081].tail(30)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
1080761,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/10/20,102353
1084101,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/11/20,103505
1087441,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/12/20,104677
1090781,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/13/20,105814
1094121,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/14/20,106987
1097461,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/15/20,107848
1100801,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/16/20,108781
1104141,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/17/20,110050
1107481,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/18/20,111383
1110821,84036081,US,USA,840,36081.0,Queens,New York,US,40.710881,-73.816847,"Queens, New York, US",12/19/20,112397


In [22]:
death_df_long.tail(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population,Date,Deaths
1179010,84056029,US,USA,840,56029.0,Park,Wyoming,US,44.521575,-109.585283,"Park, Wyoming, US",29194,1/8/21,15
1179011,84056031,US,USA,840,56031.0,Platte,Wyoming,US,42.132991,-104.966331,"Platte, Wyoming, US",8393,1/8/21,9
1179012,84056033,US,USA,840,56033.0,Sheridan,Wyoming,US,44.790489,-106.886239,"Sheridan, Wyoming, US",30485,1/8/21,17
1179013,84056035,US,USA,840,56035.0,Sublette,Wyoming,US,42.765583,-109.913092,"Sublette, Wyoming, US",9831,1/8/21,7
1179014,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,"Sweetwater, Wyoming, US",42343,1/8/21,24
1179015,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,"Teton, Wyoming, US",23464,1/8/21,4
1179016,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,"Uinta, Wyoming, US",20226,1/8/21,8
1179017,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,"Unassigned, Wyoming, US",0,1/8/21,0
1179018,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,"Washakie, Wyoming, US",7805,1/8/21,21
1179019,84056045,US,USA,840,56045.0,Weston,Wyoming,US,43.839612,-104.567488,"Weston, Wyoming, US",6927,1/8/21,4


## Merging Confirmed and Death data

In [23]:
full_table = confirmed_df_long.merge(
    right=death_df_long,
    how='left',
    on=[ 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key','Date']
)

full_table.head(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Population,Deaths
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1/22/20,0,55869.0,0.0
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",1/22/20,0,223234.0,0.0
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",1/22/20,0,24686.0,0.0
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",1/22/20,0,22394.0,0.0
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1/22/20,0,57826.0,0.0
5,84001011,US,USA,840,1011.0,Bullock,Alabama,US,32.100305,-85.712655,"Bullock, Alabama, US",1/22/20,0,10101.0,0.0
6,84001013,US,USA,840,1013.0,Butler,Alabama,US,31.753001,-86.680575,"Butler, Alabama, US",1/22/20,0,19448.0,0.0
7,84001015,US,USA,840,1015.0,Calhoun,Alabama,US,33.774837,-85.826304,"Calhoun, Alabama, US",1/22/20,0,113605.0,0.0
8,84001017,US,USA,840,1017.0,Chambers,Alabama,US,32.913601,-85.390727,"Chambers, Alabama, US",1/22/20,0,33254.0,0.0
9,84001019,US,USA,840,1019.0,Cherokee,Alabama,US,34.178060,-85.606390,"Cherokee, Alabama, US",1/22/20,0,26196.0,0.0


In [ ]:
#full_table['Date'] = pd.to_datetime(full_table['Date'])


In [24]:
ship_data = full_table['Province_State'].str.contains('Grand Princess') | full_table['Province_State'].str.contains('Diamond Princess') | full_table['Province_State'].str.contains('Northern Mariana Islands') | full_table['Province_State'].str.contains('American Samoa') |full_table['Province_State'].str.contains('Guam') | full_table['Province_State'].str.contains('Virgin Islands')

full_ship = full_table[ship_data]


In [25]:
# Removing ship data from State data

full_table = full_table[~(ship_data)]

## Group data

In [26]:
full_grouped = full_table.groupby(['Date', 'Province_State','FIPS'])['Confirmed', 'Deaths'].sum().reset_index()

full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths
1173367,9/9/20,Wyoming,56041.0,308,2.0
1173368,9/9/20,Wyoming,56043.0,111,6.0
1173369,9/9/20,Wyoming,56045.0,21,0.0
1173370,9/9/20,Wyoming,80056.0,0,0.0
1173371,9/9/20,Wyoming,90056.0,0,0.0


In [27]:
full_grouped.shape

(1173372, 5)

####  Adding new cases, new deaths by subtracting from previous day record

In [28]:
full_grouped_ = full_grouped.copy()

In [29]:
full_grouped = full_grouped_.copy()

In [30]:
def fixDate(x):
  arr = x.split('/')
  m = arr[0]
  d = arr[1]
  y = arr[2]

  if int(m) < 10:
    m = '0'+str(m)
  if int(d) < 10:
    d = '0'+str(d)
  return '20'+str(y)+'-'+m+'-'+d

In [31]:
full_grouped['Date'] = full_grouped['Date'].apply(lambda x: fixDate(x))

full_grouped.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths
0,2021-01-01,Alabama,1001.0,4239,50.0
1,2021-01-01,Alabama,1003.0,13823,169.0


In [32]:
def computeNewCases(FIPS):


  countyData = full_grouped[full_grouped['FIPS'] == FIPS]
  countyData = countyData.sort_values(by=['Date'])

  if countyData.Confirmed.max() > 0:
    temp = countyData.groupby(['Date'])['Confirmed', 'Deaths']
    temp = temp.sum().diff().reset_index()
    #print(temp)
    mask = temp['Date'] != temp['Date'].shift(1)

    #temp.loc[mask, 'Confirmed'] = np.nan
    #temp.loc[mask, 'Deaths'] = np.nan
    temp.columns = ['Date', 'New cases', 'New deaths']
    print(temp)
    countyData = pd.merge(countyData, temp, on=[ 'Date'])
    print(countyData)
    #print('********')
    # filling na with 0
    countyData = countyData.fillna(0)
    # fixing data types
    cols = ['New cases', 'New deaths']
    countyData[cols] = countyData[cols].astype('int') 
    return countyData


Below code executes for all FIPS, estimated runtime **10 minutes**

In [33]:
df = pd.DataFrame(columns=['Date','Province_State','FIPS','Confirmed','Deaths','New cases','New deaths'])

for fips in tqdm(full_grouped.FIPS.unique()):
  

  countyData = full_grouped[full_grouped['FIPS'] == fips]
  countyData = countyData.sort_values(by=['Date'])
  #print(fips,' , ',countyData.Confirmed.min())
  
  if countyData.Confirmed.max() > 0:
    temp = countyData.groupby(['Date'])['Confirmed', 'Deaths']
    temp = temp.sum().diff().reset_index()
    #print(temp)
    mask = temp['Date'] != temp['Date'].shift(1)

    # temp.loc[mask, 'Confirmed'] = np.nan
    # temp.loc[mask, 'Deaths'] = np.nan
    temp.columns = ['Date', 'New cases', 'New deaths']
    countyData = pd.merge(countyData, temp, on=[ 'Date'])
    #print(countyData)
    #print('********')
    # filling na with 0
    countyData = countyData.fillna(0)
    # fixing data types
    cols = ['New cases', 'New deaths']
    countyData[cols] = countyData[cols].astype('int')
    df = df.append(countyData)
    #break

100%|██████████| 3324/3324 [09:02<00:00,  6.13it/s]


In [34]:
print(df.shape)
df.tail(15)

(1151133, 7)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
338,2020-12-25,Wyoming,90056.0,0,0.0,0,0
339,2020-12-26,Wyoming,90056.0,0,0.0,0,0
340,2020-12-27,Wyoming,90056.0,0,0.0,0,0
341,2020-12-28,Wyoming,90056.0,0,0.0,0,0
342,2020-12-29,Wyoming,90056.0,0,0.0,0,0
343,2020-12-30,Wyoming,90056.0,0,0.0,0,0
344,2020-12-31,Wyoming,90056.0,0,0.0,0,0
345,2021-01-01,Wyoming,90056.0,0,0.0,0,0
346,2021-01-02,Wyoming,90056.0,0,0.0,0,0
347,2021-01-03,Wyoming,90056.0,0,0.0,0,0


In [35]:
df.shape

(1151133, 7)

In [36]:
df[df['FIPS'] == 56041.0].tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
348,2021-01-04,Wyoming,56041.0,1589,7.0,19,0
349,2021-01-05,Wyoming,56041.0,1597,7.0,8,0
350,2021-01-06,Wyoming,56041.0,1618,7.0,21,0
351,2021-01-07,Wyoming,56041.0,1668,8.0,50,1
352,2021-01-08,Wyoming,56041.0,1693,8.0,25,0


In [37]:
# merging new values
full_grouped = df.copy()

In [38]:
full_grouped.tail(5)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
348,2021-01-04,Wyoming,90056.0,0,0.0,0,0
349,2021-01-05,Wyoming,90056.0,0,0.0,0,0
350,2021-01-06,Wyoming,90056.0,0,0.0,0,0
351,2021-01-07,Wyoming,90056.0,0,0.0,0,0
352,2021-01-08,Wyoming,90056.0,0,0.0,0,0


In [39]:
county_population_US.head(2)

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [40]:
county_population_US['FIPS'] = county_population_US['countyFIPS']

In [46]:
county_population_US.head()

,countyFIPS,County Name,State,population,FIPS
0,0,Statewide Unallocated,AL,0,0
1,1001,Autauga County,AL,55869,1001
2,1003,Baldwin County,AL,223234,1003
3,1005,Barbour County,AL,24686,1005
4,1007,Bibb County,AL,22394,1007


In [47]:
county_population_US = county_population_US[['County Name',	'State',	'population',	'FIPS']]
county_population_US.head(2)

,County Name,State,population,FIPS
0,Statewide Unallocated,AL,0,0
1,Autauga County,AL,55869,1001


# Merge with FIPS 

In [48]:
merged = pd.merge(full_grouped,county_population_US,how='inner' ,on=['FIPS'])
print(merged.shape)

(1103831, 10)


In [49]:
merged.tail(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
1103829,2021-01-07,Wyoming,56045.0,483,0.0,4,0,Weston County,WY,6927
1103830,2021-01-08,Wyoming,56045.0,485,0.0,2,0,Weston County,WY,6927


## Visualization

In [50]:
full_grouped.head(3)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths
0,2020-01-22,Alabama,1001.0,0,0.0,0,0
1,2020-01-23,Alabama,1001.0,0,0.0,0,0
2,2020-01-24,Alabama,1001.0,0,0.0,0,0


In [51]:
merged.head(3)


,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
0,2020-01-22,Alabama,1001.0,0,0.0,0,0,Autauga County,AL,55869
1,2020-01-23,Alabama,1001.0,0,0.0,0,0,Autauga County,AL,55869
2,2020-01-24,Alabama,1001.0,0,0.0,0,0,Autauga County,AL,55869


# Export data to csv

In [52]:
import pandas as pd
import altair as alt
#full_grouped = merged
ca_df = merged[merged['Province_State'] == 'California']
ny_df = merged[merged['Province_State'] == 'New York']
tx_df = merged[merged['Province_State'] == 'Texas']

print('CA data shape: ',ca_df.shape)
print('NY data shape: ',ny_df.shape)
print('TX data shape: ',tx_df.shape)

CA data shape:  (20474, 10)
NY data shape:  (21886, 10)
TX data shape:  (89662, 10)


In [53]:

from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'CA_CovidData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
ca_df.to_csv(location+filename,index=False)



CA_CovidDataJan10.csv


In [54]:
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'NY_CovidData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
ny_df.to_csv(location+filename,index=False)

NY_CovidDataJan10.csv


In [55]:

location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'TX_CovidData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
tx_df.to_csv(location+filename,index=False)



TX_CovidDataJan10.csv


# Compute rolling Average for new cases 

In [56]:
ca_total = (
    ca_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)
ny_total = (
    ny_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)

tx_total = (
    tx_df
    .pipe(lambda x: x.assign(gains_pctg=x["New cases"]))
    .groupby(['Date','Province_State'])
    .agg({"gains_pctg": "sum"})
    .reset_index()
    .rename(columns={"gains_pctg": "New cases"})
)

In [57]:
ny_total['rolling_average'] = ny_total.iloc[:,2].rolling(window=7).mean()
ca_total['rolling_average'] = ca_total.iloc[:,2].rolling(window=7).mean()
tx_total['rolling_average'] = tx_total.iloc[:,2].rolling(window=7).mean()


In [68]:
tx_total.tail(10)

,Date,Province_State,New cases,rolling_average
343,2020-12-30,Texas,22004,14641.000000
344,2020-12-31,Texas,17809,15879.714286
345,2021-01-01,Texas,10172,16859.285714
346,2021-01-02,Texas,18225,18689.428571
347,2021-01-03,Texas,15705,16813.142857
348,2021-01-04,Texas,14260,17618.285714
349,2021-01-05,Texas,33027,18743.142857
350,2021-01-06,Texas,20626,18546.285714
351,2021-01-07,Texas,25292,19615.285714
352,2021-01-08,Texas,13308,20063.285714


# Visualizations

In [62]:
base_ca = alt.Chart(ca_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)

base_ny = alt.Chart(ny_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)
base_tx = alt.Chart(tx_total).mark_bar().encode(
    x='monthdate(Date):O',
).properties(
    width=500
)


In [66]:
red = alt.value("#f54242")
##Ca data
#base_ca.encode(y='Confirmed').properties(title='Total Confirmed') | base_ca.encode(y='Deaths',color = red).properties(title='Total deaths')
base_ca.encode(y='New cases').properties(title='CA State- New cases')  | base_ca.encode(y='rolling_average',color = red).properties(title='Rolling Average')

alt.HConcatChart(...)

In [65]:
#base_ny.encode(y='Confirmed').properties(title='Total Confirmed') | base_ny.encode(y='Deaths',color = red).properties(title='Total deaths')

base_ny.encode(y='New cases').properties(title='NY state - New cases')  | base_ny.encode(y='rolling_average',color = red).properties(title='Rolling Average')


alt.HConcatChart(...)

In [64]:
red = alt.value("#f54242")
##Ca data
#base_ca.encode(y='Confirmed').properties(title='Total Confirmed') | base_ca.encode(y='Deaths',color = red).properties(title='Total deaths')
base_tx.encode(y='New cases').properties(title='TX State- New cases') | base_tx.encode(y='rolling_average',color = red).properties(title='Rolling Average')

alt.HConcatChart(...)

## Reference

* https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6